In [889]:
from bs4 import BeautifulSoup
import requests
import re
import os

In [727]:
def save_content(resp, filename):
    with open (filename, 'wb') as file:
        file.write(resp.content)
    

In [776]:
def get_user_info(username):
    
    #Get name, organisation, followers, place and number of public repositories from github by name
    
    full_path = 'https://github.com/' + username
    req = requests.get(full_path)
    get_closer = BeautifulSoup(req.content)
    full_name = get_closer.find('span',
                               class_="p-name vcard-fullname d-block overflow-hidden").text.strip()
    org = get_closer.find('span', class_="p-org").text
    place = get_closer.find('span', class_="p-label").text
    followers = get_closer.find('span', class_="text-bold color-fg-default").text
    repo_num = get_closer.find('span', class_="Counter").text
    res = {'Name':full_name,
          'Organisation':org,
          'Place':place,
          'Followers':followers,
          'Repositories':repo_num}
    return res

In [783]:
#Check it!
get_user_info('polarsong')

{'Name': 'Alya Mikhailova',
 'Organisation': 'Center For Mitochondrial Functional Genomics IKBFU',
 'Place': 'Kaliningrad',
 'Followers': '9',
 'Repositories': '1'}

In [837]:
def get_user_repositories(username):
    #Get all public repos for user into list of dicts
    
    full_path = 'https://github.com/' + username
    req = requests.get(full_path, params = {'tab':'repositories'})
    get_closer = BeautifulSoup(req.content)
    
    res_list = []
    all_repo = get_closer.find_all('a', itemprop="name codeRepository")
    all_progleng = get_closer.find_all('span', itemprop="programmingLanguage")
    
    for repo,leng in zip(all_repo,all_progleng):
        cur_dict = {'username':username,
                   'Repository':repo.text.strip(),
                   'Language':leng.text}
        res_list.append(cur_dict)
        
    return res_list

In [839]:
#Check 2
get_user_repositories('DIliushchenko')

[{'username': 'DIliushchenko',
  'Repository': 'BI_2021_Python',
  'Language': 'Jupyter Notebook'},
 {'username': 'DIliushchenko',
  'Repository': 'BI_ML_2021',
  'Language': 'Jupyter Notebook'},
 {'username': 'DIliushchenko', 'Repository': 'Main_Scripts', 'Language': 'R'},
 {'username': 'DIliushchenko',
  'Repository': 'BI_Stat_2021',
  'Language': 'HTML'},
 {'username': 'DIliushchenko',
  'Repository': 'Runs-of-Homozygosity',
  'Language': 'R'}]

In [899]:
def list_repository_contents(username, repository, repository_path = ''):
    
    '''
    Function to get all objects from repository path
    Use repository path with form '/tree/branch/path_to_folder' to get correct result
    '''
    
    full_path = 'https://github.com/' + username + '/' + repository + repository_path
    req = requests.get(full_path)
    ser_files = BeautifulSoup(req.content)
    result = []
    for obj in ser_files.find_all('a', class_="js-navigation-open Link--primary"):
        result.append(obj.text)
    return result

In [874]:
# Check 3
list_repository_contents('DIliushchenko','Runs-of-Homozygosity','/tree/CoverageRoh/Body/1Raw')

['ROH_ansar',
 'RecombMap',
 'PleaseDeleteme',
 'TestWind.txt',
 'gencode.v25.annotation.gtf.Genes.Shet.pLI.FIS.RVIS.GHIS.KnKs.GC.BrainSpecificRanking.Branch']

In [923]:
def download_file(username, repository, remote_file_path, local_file_path = ''):
    
    '''
    Function to download object from repository path
    Use remote_file_path with form '/branch/path_to_file' to get correct result
    If your input isn't a file, function rises error
    '''
    full_path = 'https://raw.githubusercontent.com/' + username + '/' + repository + remote_file_path
    req = requests.get(full_path)
    if req.status_code == 400 or req.status_code == 404:
        raise ValueError("It's not a file! Or file doesn't exist")
    else:
        name_of_file = remote_file_path.split('/')[-1]
        open(os.path.join(local_file_path, name_of_file), 'wb').write(req.content)

In [922]:
file_path = '/CoverageRoh/Body/1Raw/gencode.v25.annotation.gtf.Genes.Shet.pLI.FIS.RVIS.GHIS.KnKs.GC.BrainSpecificRanking.Branch'

In [924]:
# Check 4
download_file('DIliushchenko','Runs-of-Homozygosity', file_path)

In [917]:
requests.get('https://raw.githubusercontent.com/DIliushchenko')

<Response [400]>